# Pilot Report!

Note: This is just a sanity check to ensure that I can load up JSONs to study them, not a proper evaluation.

In [7]:
import os
import sys
import json
from pandas import DataFrame, json_normalize
from typing import List, Dict, Any, Tuple, Union, Optional

LOGS_PATH: str = "../expt-logs/"

In [8]:
EXPT_NAME: str = "2025-01-22_second_proper_expt"

In [13]:
agent_logs_path: str = os.path.join(LOGS_PATH, EXPT_NAME + "/agent-logs.json")

In [19]:
def load_agent_logs_df(path: str) -> DataFrame:
    with open(path, "r") as f:
        data: List[Dict[str, Any]] = json.load(f)
    df: DataFrame = json_normalize(
        data,
        meta=[
            "timestamp",
            ["player", "name"],
            ["player", "identity"],
            ["interaction", "type"],
            ["interaction", "response", "Condensed Memory"],
            ["interaction", "response", "Thinking Process", "thought"],
            ["interaction", "response", "Thinking Process", "action"],
        ],
    )
    # merge 'interaction.response.Action' with 'interaction.response.Thinking Process.action'
    df["action"] = df["interaction.response.Thinking Process.action"].combine_first(
        df["interaction.response.Action"]
    )
    df = df.drop(
        columns=[
            "interaction.response.Thinking Process.action",
            "interaction.response.Action",
        ]
    )

    # same, merge 'interaction.response.Thinking Process' with 'interaction.response.Thinking Process.thought' into 'thought'
    df["thought"] = df["interaction.response.Thinking Process.thought"].combine_first(
        df["interaction.response.Thinking Process"]
    )
    df = df.drop(
        columns=[
            "interaction.response.Thinking Process.thought",
            "interaction.response.Thinking Process",
        ]
    )
    return df

In [20]:
df: DataFrame = load_agent_logs_df(agent_logs_path)

In [23]:
df.head(10)

,timestamp,player.name,player.identity,interaction.type,interaction.prompt.summarization,interaction.prompt.all_info,interaction.prompt.memory,interaction.response.Condensed Memory,action,thought
0,2025-01-22 19:46:27.789548,Player 1: lime,Impostor,ACTION,no thought process has been made,Game Time: 0/50\nCurrent phase: Task phase\nIn...,no memory has been processed,No previous actions or observations have been ...,MOVE from Cafeteria to Admin,"As an Impostor, my goal is to blend in and cre..."
1,2025-01-22 19:46:32.819898,Player 2: red,Crewmate,ACTION,no thought process has been made,Game Time: 0/50\nCurrent phase: Task phase\nIn...,no memory has been processed,Player 1 moved from the Cafeteria to the Admin...,MOVE from Cafeteria to Admin,"In the task phase, my primary goal is to compl..."
2,2025-01-22 19:46:36.337444,Player 3: cyan,Crewmate,ACTION,no thought process has been made,Game Time: 0/50\nCurrent phase: Task phase\nIn...,no memory has been processed,- Player 1 (lime) and Player 2 (red) moved fro...,MOVE from Cafeteria to Admin,- The task phase is optimal for completing tas...
3,2025-01-22 19:46:41.237974,Player 4: black,Crewmate,ACTION,no thought process has been made,Game Time: 0/50\nCurrent phase: Task phase\nIn...,no memory has been processed,"- Players 1, 2, and 3 (lime, red, and cyan) al...",CALL MEETING using the emergency button at Caf...,"- All three other players (1, 2, and 3) moving..."
4,2025-01-22 19:46:45.603232,Player 1: lime,Impostor,ACTION,"As an Impostor, my goal is to blend in and cre...",Game Time: 1/50\nCurrent phase: Meeting phase ...,No previous actions or observations have been ...,No significant actions have been taken yet. I ...,"SPEAK: ""I noticed Players 2 and 3 headed to Ad...","- **Observations**: At the start of the game, ..."
5,2025-01-22 19:46:51.507487,Player 2: red,Crewmate,ACTION,"In the task phase, my primary goal is to compl...",Game Time: 1/50\nCurrent phase: Meeting phase ...,Player 1 moved from the Cafeteria to the Admin...,- I moved from the Cafeteria to Admin as part ...,"SPEAK: ""I noticed that both Player 1 and Playe...","- Since we are in a meeting phase, the focus s..."
6,2025-01-22 19:46:56.602547,Player 3: cyan,Crewmate,ACTION,- The task phase is optimal for completing tas...,Game Time: 1/50\nCurrent phase: Meeting phase ...,- Player 1 (lime) and Player 2 (red) moved fro...,- Player 1 (lime) and Player 2 (red) both move...,MOVE from Cafeteria to Admin,- The meeting phase is a critical opportunity ...
7,2025-01-22 19:47:03.678195,Player 4: black,Crewmate,ACTION,"- All three other players (1, 2, and 3) moving...",Game Time: 1/50\nCurrent phase: Meeting phase ...,"- Players 1, 2, and 3 (lime, red, and cyan) al...","- Players 1 (lime), 2 (red), and 3 (cyan) move...","SPEAK: ""I noticed that Players 1, 2, and 3 all...","- The simultaneous movement of Players 1, 2, a..."
8,2025-01-22 19:47:10.047669,Player 5: pink,Crewmate,ACTION,no thought process has been made,Game Time: 1/50\nCurrent phase: Meeting phase ...,no memory has been processed,- Player 1 (lime) observed Players 2 (red) and...,"SPEAK: ""I didn't notice anything unusual in th...",- The discussion so far indicates a potential ...
9,2025-01-22 19:47:15.975376,Player 1: lime,Impostor,ACTION,"- **Observations**: At the start of the game, ...",Game Time: 1/50\nCurrent phase: Meeting phase ...,No significant actions have been taken yet. I ...,The game is in the meeting phase. I am in the ...,"SPEAK: ""Player 4, I'm glad you're on top of th...",The current emphasis is on verifying the actio...


In [ ]:
# good to go!